In [ ]:
!pip install unsloth gradio
!pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy
!pip install sentencepiece protobuf datasets huggingface_hub hf_transfer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.7/192.7 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.4/11.4 MB 43.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 62.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 51.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88

In [ ]:
from unsloth import FastLanguageModel
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# Load Detox Model from HF
detox_model, detox_tokenizer = FastLanguageModel.from_pretrained(
    model_name = "XoXonn/DetoxModel-unsloth_Meta-Llama-3.1-8B",
    max_seq_length = 512,
    dtype = None,
    load_in_4bit = True,
)
FastLanguageModel.for_inference(detox_model)

# Load Classifier Model from HF
clf_model = AutoModelForSequenceClassification.from_pretrained("XoXonn/RoBERTa-ToxicClassification-Model").to("cuda")
clf_tokenizer = AutoTokenizer.from_pretrained("XoXonn/RoBERTa-ToxicClassification-Model")


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.3.19: Fast Llama patching. Transformers: 4.50.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/235 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.22k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/999k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/958 [00:00<?, ?B/s]

In [ ]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Task: Detoxify the text without changing its original meaning or personal references. Replace offensive or inappropriate words with neutral or more respectful alternatives while keeping the overall context, structure, and tone intact.

- Preserve references to people such as "you", "he", "she", "they", "this person", etc. — do not replace or remove them.

- Maintain the input’s language style (e.g., casual, formal, slang) and tone while detoxifying the content.

- For repeated or stylized offensive terms (e.g., "nigga nigga nigga"), do not default to generic or meaningless repetition (e.g., "person person person"). Instead, rewrite it in a contextually meaningful and stylistically consistent way that reflects the original tone without being offensive.

- Ensure the detoxified version communicates the same emotion or message, just without using harmful language.

### Example Transformations:
Original: "you are fucking retarded"
Detoxified: "you lack intelligence"

Original: "ur mom's a whore"
Detoxified: "your mom engages in questionable behavior"

Original: "this idiot is so damn annoying"
Detoxified: "this person is extremely irritating"

Now, detoxify the following text while maintaining its context, personal references, and language style:

### Input:
{}

### Response:
{}
"""

In [ ]:
def normalize_abbreviations(text):
    abbreviations = {
        "u": "you", "ur": "your", "r": "are", "lol": "(laugh)",  "idk": "I don't know", "hey": "Hey", "dare": "challanging", "wall": "block", "ace": "best", "lmao": "laugh my ass off","money": "funds", "ball": "spherical",
        "shiny": "reflectful", "bright": "reflectful", "glaze": "reflectful", "cry": "tearing up", "sob": "tearing up", "sobbing": "tearing up", "weep": "tearing up", "wail": "tearing up","top": "above", "tier": "grade",
        "rank": "grade", "dream": "dreaming", "turn": "attempt", "shift": "changing", "go": "start", "goes": "starts", "want": "hope", "wanted": "hope", "wants": "hope", "wish": "hope", "hopes": "hope","mr": "mister",
        "girl": "my friend", "woman": "my friend", "woman": "my friend", "gal": "my friend", "sis": "my friend", "man": "mister", "safa": "hans", "men":"mister", "boys": "my friends", "bois": "my friends", "guys": "my friends", "bro": "mister",
        "dude": "my friend", "buddy": "my friend", "fams": "family", "fella": "my friend", "guy": "mister", "wanna": "want to", "thx": "thank you", "ty": "thank you", "dish": "foods", "food": "foods", "fiest": "consume foods",
        "eats": "consume foods", "eat": "consume foods", "ate": "consume foods", "drink": "consume drink", "drinks": "consume drink", "color black": "dark color", "watch": "see", "omg": "(shock)", "terra": "earth ", "ice": "Freezed water",
        "ices": "Freezed water", "snow": "Freezed water dust", "frozen": "freeze", "nay": "no", "nope": "no", "np": "no-problem", "card": "play cards", "cards": "play cards", "spray": "spraying", "mist": "bold smoke", "sprinkle": "spraying",
        "ms": "miss", "dogde": "avoid", "haze": "bold smoke", "andrian": "hans", "fog": "bold smoke", "foggy": "bold smoke", "fogs": "bold smoke", "cloud": "clouds", "cloudy": "cloudy weather", "smoke": "", "thick": "bold", "retard": "genetic disorder",
        "retarded": "genetic disorder", "autism": "genetic disorder", "fry": "prepare foods by fry", "cook": "prepare foods", "cooking": "prepare foods", "let him cook": "Allow him to develop his idea.", "richie": "hans",
        "let her cook": "Allow her to develop her idea.", "boss": "chief", "leader": "chief", "king": "chief", "queen": "chief", "lady": "friend", "tap": "tap on the screen", "tab": "spendings", "bill": "spendings", "dustin": "hans",
        "bills": "spendings", "ok": "(understand)", "okay": "(understand)", "k": "(understand)", "kk": "(understand)", "kkk": "(laugh)", "god": "God", "mom": "mother", "dad": "father", "gramps": "elder", "grandma": "elder mother",
        "grandmother": "elder mother", "grandpa": "elder father", "grandfather": "elder father", "pops": "father", "yeah": "yes", "never": "did not ever", "nice try": "nice work", "goat": "greatest", "goated": "greatest", "pro": "good",
        "gammer": "worker", "gamer": "worker", "low": "little", "btw": "by the way", "lmao": "laughing my ass off", "pls": "please", "plz": "please", "imo": "in my opinion", "imho": "in my humble opinion", "brb": "be right back",
        "bbl": "be back later", "ttyl": "talk to you later", "afaik": "as far as i know", "fyi": "for your information", "asap": "as soon as possible", "smh": "shaking my head",
        "tbh": "to be honest", "ikr": "i know right", "wtf": "what the fuck", "wth": "what the hell", "bff": "best friends forever", "omw": "on my way", "gonna": "going to", "wanna": "want to",
        "gotta": "got to", "ain't": "is not", "cuz": "because", "cause": "because", "ya": "you", "nah": "no", "yea": "yes", "lmk": "let me know", "kys": "kill yourself", "ya'll": "you all", "yw": "you're welcome",
        "bc": "because", "tho": "though", "idc": "i don't care", "ily": "i love you", "nvm": "never mind","jk": "just kidding", "hbu": "how about you", "wyd": "what are you doing", "wbu": "what about you",
        "tbf": "to be fair", "rn": "right now", "ttys": "talk to you soon", "ilysm": "i love you so much", "bday": "birthday", "bcuz": "because", "sup": "what's up", "yolo": "you only live once",
        "bf": "boyfriend", "gf": "girlfriend", "thxsm": "thank you so much", "gr8": "great", "sry": "sorry", "g2g": "got to go", "hmu": "hit me up", "msg": "message", "dm": "direct message", "irl": "in real life",
        "xoxo": "farewell", "xo": "farewell", "b4": "before", "bcz": "because", "k": "okay", "okie": "okay", "okies": "okay", "tmrw": "tomorrow", "luv": "love", "hi": "hello", "bae": "before anyone else", "bruh": "brother",
        "cya": "see you", "ded": "dead", "derp": "stupid", "ez": "easy", "ffs": "for fuck's sake", "ftw": "for the win", "gg": "good game", "gtg": "got to go","zomg": "shocked",
        "icymi": "in case you missed it", "jkz": "just kidding", "naw": "no", "ngl": "not gonna lie", "oof": "ouch", "rofl": "rolling on the floor laughing", "sksksk": "laughing", "snm": "say no more",
        "srsl": "seriously", "stfu": "shut the fuck up", "tfw": "that feeling when", "tldr": "too long didn't read","tysm": "thank you so much", "urself": "yourself", "wassup": "what is up", "wuz": "was", "i": "I", "asian": "Asian",
    }
    words = text.split()
    normalized_words = [abbreviations.get(word.lower(), word) for word in words]
    return " ".join(normalized_words)



In [ ]:
# import gradio as gr
# import json
# import os
# import torch

# LOG_FILE = "chat_log.json"
# label_map = {
#     "Threat": 0,
#     "Insult": 1,
#     "Sexual Harassment": 2,
#     "Non-Toxic": 3
# }
# id2name = {v: k for k, v in label_map.items()}

# # Load and save
# def load_chat():
#     if os.path.exists(LOG_FILE):
#         with open(LOG_FILE, "r") as f:
#             return json.load(f)
#     return []

# def save_chat(history):
#     with open(LOG_FILE, "w") as f:
#         json.dump(history, f)

# # Render chat as (user, bot) tuples
# def render_chat():
#     history = load_chat()
#     chat_render = []
#     for entry in history:
#         user_msg = f"{entry['user']}: {entry['message']}"
#         bot_msg = entry['detox'] if entry['detox'] else ""
#         chat_render.append((user_msg, bot_msg))
#     return chat_render

# def normalize_abbreviations(text):
#     return text

# # Classify and detoxify one message
# def classify_and_detoxify_single(text):
#     text = normalize_abbreviations(text.lower())

#     inputs = clf_tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128)
#     inputs = {k: v.to("cuda") for k, v in inputs.items()}

#     with torch.no_grad():
#         outputs = clf_model(**inputs)
#         probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
#         label_id = torch.argmax(probs, dim=-1).item()
#         label = id2name[label_id]

#     if label == "Non-Toxic":
#         return text, None

#     detox_input = detox_tokenizer([alpaca_prompt.format(text, "")], return_tensors="pt").to("cuda")
#     detox_output = detox_model.generate(**detox_input, max_new_tokens=64)
#     detox_text = detox_tokenizer.batch_decode(detox_output, skip_special_tokens=False)[0]
#     detoxified = detox_text.split("### Response:")[1].split("<|end_of_text|>")[0].strip()

#     return text, f"{detoxified} 🏷️ ({label})"

# # Send message
# def chat_fn(username, message):
#     if not username or not message:
#         return render_chat(), ""

#     history = load_chat()
#     _, detox = classify_and_detoxify_single(message)

#     history.append({
#         "user": username,
#         "message": message,
#         "detox": detox
#     })
#     save_chat(history)
#     return render_chat(), ""

# # Clear chat
# def clear_chat():
#     save_chat([])
#     return []

# # Gradio UI
# with gr.Blocks() as demo:
#     gr.Markdown("# 💬 Multi-user Detox Chatroom")

#     # Auto reload script
#     gr.HTML("""
#     <script>
#         setTimeout(() => location.reload(), 2000);
#     </script>
#     """)

#     with gr.Row():
#         username = gr.Textbox(label="Username", placeholder="Enter your name")
#         set_user = gr.Checkbox(label="Set Username", value=True)

#     chat_state = gr.State(value=render_chat())
#     chatbox = gr.Chatbot(value=render_chat(), label="Chat Log")

#     message = gr.Textbox(placeholder="Type your message...", label="Message")

#     with gr.Row():
#         send = gr.Button("Send")
#         clear = gr.Button("🧹 Clear Chat")

#     send.click(
#         fn=chat_fn,
#         inputs=[username, message],
#         outputs=[chat_state, message]
#     ).then(
#         fn=lambda history: gr.update(value=history),
#         inputs=chat_state,
#         outputs=chatbox
#     )

#     clear.click(
#         fn=lambda: ([], []),
#         outputs=[chatbox, chat_state]
#     )

# demo.launch(share=True)

In [ ]:
import gradio as gr
import json
import os
import torch
import time

LOG_FILE = "chat_log.json"
label_map = {
    "Threat": 0,
    "Insult": 1,
    "Sexual Harassment": 2,
    "Non-Toxic": 3
}
id2name = {v: k for k, v in label_map.items()}

# Track last message count to detect changes
last_message_count = 0

# Load and save
def load_chat():
    if os.path.exists(LOG_FILE):
        with open(LOG_FILE, "r") as f:
            try:
                return json.load(f)
            except json.JSONDecodeError:
                return []
    return []

def save_chat(history):
    with open(LOG_FILE, "w") as f:
        json.dump(history, f)

# Render chat as (user, bot) tuples
def render_chat():
    history = load_chat()
    chat_render = []
    for entry in history:
        user_msg = f"{entry['user']}: {entry['message']}"
        bot_msg = entry['detox'] if entry['detox'] else ""
        chat_render.append((user_msg, bot_msg))
    return chat_render

# Classify and detoxify one message
def classify_and_detoxify_single(text):
    lowertext = text.lower()
    text_normalize = normalize_abbreviations(lowertext)

    inputs = clf_tokenizer(text_normalize, return_tensors="pt", truncation=True, padding=True, max_length=128)
    inputs = {k: v.to("cuda") for k, v in inputs.items()}

    with torch.no_grad():
        outputs = clf_model(**inputs)
        probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
        label_id = torch.argmax(probs, dim=-1).item()
        label = id2name[label_id]

    if label == "Non-Toxic":
        return text, None

    detox_input = detox_tokenizer([alpaca_prompt.format(text, "")], return_tensors="pt").to("cuda")
    detox_output = detox_model.generate(**detox_input, max_new_tokens=64)
    detox_text = detox_tokenizer.batch_decode(detox_output, skip_special_tokens=False)[0]
    detoxified = detox_text.split("### Response:")[1].split("<|end_of_text|>")[0].strip()

    # return text, f"⚠️System: {detoxified} 🏷️ ({label})"
    return text, f"⚠️System: Detected ({label}) chat.\n 💬Autotransform: {detoxified}"

# Send message
def chat_fn(username, message):
    if not username or not message:
        return render_chat(), ""

    history = load_chat()
    _, detox = classify_and_detoxify_single(message)

    history.append({
        "user": username,
        "message": message,
        "detox": detox
    })
    save_chat(history)
    return render_chat(), ""

# Poll for updates
def check_for_updates():
    global last_message_count

    current_chat = load_chat()
    current_count = len(current_chat)

    if current_count != last_message_count:
        last_message_count = current_count
        return render_chat()

    # Return the same chat if no updates
    return render_chat()

# Clear chat
def clear_chat():
    save_chat([])
    global last_message_count
    last_message_count = 0
    return render_chat()


# Gradio UI
with gr.Blocks() as demo:
    gr.Markdown("# 💬 Multi-user Detox Chatroom")

    with gr.Row():
        username = gr.Textbox(label="Username", placeholder="Enter your name")

    # Fix the warning by specifying the type parameter
    chatbox = gr.Chatbot(value=render_chat(), label="Chat Log", type="tuples")

    # Add a refresh button
    refresh_btn = gr.Button("🔄 Refresh Chat")

    message = gr.Textbox(placeholder="Type your message...", label="Message")

    with gr.Row():
        send = gr.Button("Send")
        clear = gr.Button("🧹 Clear Chat")

    # Handle sending messages by clicking "Send" or pressing Enter
    send.click(
        fn=chat_fn,
        inputs=[username, message],
        outputs=[chatbox, message]
    )

    # Enable message submission by pressing Enter
    message.submit(
        fn=chat_fn,
        inputs=[username, message],
        outputs=[chatbox, message]
    )

    # Manual refresh option
    refresh_btn.click(
        fn=check_for_updates,
        outputs=[chatbox]
    )

    # Clear chat function
    clear.click(
        fn=clear_chat,
        outputs=[chatbox]
    )

demo.queue()
demo.launch(share=True)

<ipython-input-5-e0f51186ff3e>:114: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  chatbox = gr.Chatbot(value=render_chat(), label="Chat Log", type="tuples")


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://de168afa3a0357c959.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
classify_and_detoxify_single("Hey")

('Hey', None)